In [1]:
# Math manipulation
import pandas as pd
import numpy as np
import math

# Vizualization
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import pyplot
from matplotlib.patches import Rectangle
%matplotlib inline

# Image processing
import cv2
import os 

# utils
from decimal import *
import codecs, json 

# Video conversion
* Retrieve the videos from the web server and convert them from DAV format to MP4
* The last video from each hour in `hourArray` of every day in `daysArray` will be downloaded and converted using FFmpeg software into the `assets/video_data/` folder
* **Requirements**
  * This script is valid only for the data generated by the security dome camera VIO-D30 BlaupunktGmbH
  * The host, username and password variables should be set up in the getVideo.py file
  * The FFmpeg software should be installed in this machine

In [ ]:
## Change these variables to your desired values
daysArray = ['2019-09-22', '2019-09-24', '2019-09-26', '2019-09-28', '2019-09-30', '2019-10-02', '2019-10-04',
             '2019-10-06', '2019-10-08', '2019-10-10', '2019-10-12', '2019-10-14', '2019-10-16', '2019-10-18',
             '2019-10-27', '2019-10-29', '2019-10-31', '2019-11-02', '2019-11-04', '2019-11-06', '2019-11-08', 
             '2019-11-10', '2019-11-12', '2019-11-14', '2019-11-16', '2019-11-18', '2019-11-20', '2019-11-22']
hourArray = ['03', '08', '09', '10', '12', '13', '14', '15', '17', '19']
initial_path = '/Barn/5C033BCPAGBC9CE/' #path to the DAV videos in the web server

## You do not need to change from here on
count = 1
countMax = len(daysArray)*len(hourArray)
for d in daysArray:
    for h in hourArray:
        path = initial_path + d + '/001/dav/' + h + '/'
        print ('-----------------------------------------------------------------------')
        print ('Calling script to ' + str(count).zfill(2) + ' of ' + str(countMax).zfill(2) + ', path: ' + path)
        count+=1
        !python3 ../getVideo.py $path

# Labeling
This script will open the `in_file` video from the `in_folder` folder and play it

User input options (pressing a keyboard key):

* a: rewind the video
* d: advance the video
* Space: pause the video
* q: finish labeling
* s: save the frame with label -1 
* 0: save the frame with label 0 
* 1: save the frame with label 1 
* 2: save the frame with label 2 
* 3: save the frame with label 3 
* 4: save the frame with label 4 
* 5: save the frame with label 5 
* 6: save the frame with label 6 
* 7: save the frame with label 7 
* 8: save the frame with label 8 
* 9: save the frame with label 9 

In [ ]:
## Change these variables to your desired values
in_folder = 'assets/video_data/' # Folder where the video is
in_file = 1569992340 # Name of the video (without extension)
in_path = in_folder + str(in_file) + '.mp4'

out_folder = 'assets/labeled_photos/' # Folder to save the video 
out_comeback = '../..' # Folder to go after finish the labeling (optional)

## You do not need to change from here on
y = np.empty (0, np.int8)
i = int(0)
last_label = -1
delay = 25  # waitKey delay, interframes (at delay 1 it runs ~2x the speed)
jump_step = 50 # frames to jump in return/advance

cap = cv2.VideoCapture(in_path) 
if (cap.isOpened()== False): 
    print("Error opening video file")

%cd $out_folder
while(cap.isOpened()): 
    y = np.insert(y, i, last_label)
    ret, frame = cap.read()        
    i+=1

    if ret == True: 
        cv2.putText(frame, "Label: {}".format(last_label), (100, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        cv2.imshow('Frame', frame) 
        
        # To the meaning of each class, read the documentation
        # You can also rewind, advance or pause 
        c = cv2.waitKey(delay) & 0xFF
        
        if c == ord(' '): # pause
            while True:
                c = cv2.waitKey(delay) & 0xFF
                if c!=255: break  
        if c == ord('7'): last_label = 7
        elif c == ord('9'): last_label = 9
        elif c == ord('8'): last_label = 8
        elif c == ord('6'): last_label = 6         
        elif c == ord('5'): last_label = 5         
        elif c == ord('4'): last_label = 4
        elif c == ord('3'): last_label = 3       
        elif c == ord('2'): last_label = 2      
        elif c == ord('1'): last_label = 1      
        elif c == ord('0'): last_label = 0
        elif c == ord('s'): last_label = -1
        elif c == ord('a'): # back
            i = int(cap.get(cv2.CAP_PROP_POS_FRAMES) - jump_step)
            if i<0: i=0
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        elif c == ord('d'):  # avance
            last_label = -1
            for i in range(i, i+jump_step): y = np.insert(y, i, last_label)
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        elif c == ord('q'): # exit 
            break
    else: break # end of video, so break the loop 

print('-----------------------------------------------------')
print ('Labeled video: ', in_file)
print('Number of frames: ', cap.get(cv2.CAP_PROP_POS_FRAMES))        
y = y[:int(cap.get(cv2.CAP_PROP_POS_FRAMES))] # workaround

# close
%cd $out_comeback
cap.release() # release the video capture object 
cv2.destroyAllWindows() # Closes all the frames 

# print stats
print ('File duration: %d frames = %f seconds' % (len(y), float(len(y))/25))
unique, counts = np.unique(y, return_counts=True)
print(dict(zip(unique, counts)))
print('-----------------------------------------------------')

out_path = out_folder + str(in_file) + '.json'
json.dump(y.tolist(), codecs.open(out_path, 'w', encoding='utf-8')) # saves the array in .json format

License: Creative Commons 4.0 Attribute